# Prikupljanje podataka

Preuzimanje tekstova pesama sa [Tekstovi.net](https://tekstovi.net/2,0,0.html) koristeći *Web Crawling* i *Web Scraping* tehnike uz oslonac na *Selenium*.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from fuzzywuzzy import fuzz

In [ ]:
target_artists = {
    'folk': ['ceca', 'aca lukas', 'lepa brena', 'dara bubamara', 'seka aleksic', 'dragana mirkovic', 'mile kitic',
             'mitar miric', 'toma zdravkovic', 'lepa lukic', 'silvana armenulic', 'semsa suljakovic', 'ahmedovski',
             'indira radic', 'sasa matic'],
    'rock': ['generacija 5', 'kerber', 'bijelo dugme', 'azra', 'bajaga', 'galija', 'parni valjak', 'riblja corba'],
    'pop': ['funky g', 'jelena tomasevic', 'marija serifovic', 'jelena karleusa', 'zeljko joksimovic', 'severina',
            'bekvalac', 'sasa kovacevic', 'luna']
}

In [ ]:
# definisanje pretraživača i pozicioniranje na početnu stranu
driver = webdriver.Firefox()
driver.get("https://tekstovi.net")

print("\nPreuzimanje linkova iz menija slova...")
first_letter_links = [link.get_attribute("href") for link in
                      driver.find_elements(By.XPATH, "//*[@id='meni_slova']//td/a")]
print(f"\nPreuzeti linkovi: {first_letter_links}")

print("\nPreuzimanje izvođača...")
artists = {}
for first_letter_link in first_letter_links:
    driver.get(first_letter_link)
    for artist in driver.find_elements(By.XPATH, "//*[@class='artLyrList']/a"):
        artists[artist.text] = artist.get_attribute('href')
print(f"\nPreuzeti izvođači: {artists}")

print("\nPreuzimanje tekstova pesama za preuzete izvođače...")
for genre in target_artists.keys():
    print(f"\n\n\nPreuzimanje tekstova za žanr: {genre}")
    
    for artist_name in target_artists[genre]:
        print(f"\nPreuzimanje tekstova za izvođača: {artist_name}")
        
        for known_name in artists.keys():
            if fuzz.ratio(artist_name.lower(), known_name.lower()) > 95:
                # otvaranje stranice izvođača i preuzimanje tekstova
                print("Započeto: " + known_name)
                driver.get(artists[known_name])
                song_links = [link.get_attribute('href') for link in
                              driver.find_elements(By.XPATH, "//*[@class='artLyrList']/a")]
                
                # preuzimanje pesmu po pesmu
                for song_link in song_links:
                    driver.get(song_link)
                    song_name = driver.find_element(By.XPATH, "//h2[@class='lyricCapt']").text

                    # izdvajanje teksta
                    lyric_text = ""
                    for lyric_part in driver.find_elements(By.XPATH, "//*[@class='lyric']"):
                        lyric_text += lyric_part.text + '\n\n'
                    
                    # upis teksta u fajl
                    with open('./data/' + genre + '/' + known_name + ' - ' + song_name + '.txt', 'w') as output_stream:
                        output_stream.write(lyric_text)
                print("Završeno: " + known_name)

# zatvaranje pretraživača
driver.close()